In [ ]:
import os
import pandas as pd

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

In [ ]:
import bnlearn as bn

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = ['Arial Unicode Ms']
# plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']

In [ ]:
# This is from TDAv2
combined_data = pd.read_csv('../ComputedData/ForModel/combined_data_with_hotspot.csv')
combined_data.shape

In [ ]:
select_group = [
    # 號誌
    '號誌-號誌種類名稱', '號誌-號誌動作名稱',

    # 車道劃分
    '車道劃分設施-分道設施-快車道或一般車道間名稱',
    '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',

    # 大類別
    '事故類型及型態子類別名稱', '車道劃分設施-分向設施子類別名稱',
    '道路型態子類別名稱',

    # 其他
    '速限-第1當事者',
    '道路類別-第1當事者-名稱',

    # 設施
    'youbike_100m_count',
    
    # 駕駛、行人行為
    '肇因研判子類別名稱-主要',

    'COUNTYNAME'
    ]

data = combined_data[select_group].copy()
data['facility'] = data[['youbike_100m_count']].apply(lambda row: '1' if (row > 0).any() else '0', axis=1)
data.drop(columns=['youbike_100m_count'], inplace=True)

max_speed = data['速限-第1當事者'].max()
bins = range(0, int(max_speed) + 11, 10)

data['速限-第1當事者'] = pd.cut(
    data['速限-第1當事者'],
    bins=bins,
    right=False, 
    include_lowest=True,
    labels=[f"{i}-{i+9}" for i in bins[:-1]]
)

data0 = data[data['facility'] == '0']
data1 = data[data['facility'] == '1']

In [ ]:
parent = [
    '號誌-號誌種類名稱', '號誌-號誌動作名稱','車道劃分設施-分道設施-快車道或一般車道間名稱', '車道劃分設施-分道設施-快慢車道間名稱',
    '車道劃分設施-分道設施-路面邊線名稱', '車道劃分設施-分向設施子類別名稱', '道路型態子類別名稱',
    '速限-第1當事者', '道路類別-第1當事者-名稱', 'facility', 'COUNTYNAME'
    ]
cause = ['肇因研判子類別名稱-主要']
result = ['事故類型及型態子類別名稱']

white_list = [
    ('速限-第1當事者', '肇因研判子類別名稱-主要'),
    ('道路類別-第1當事者-名稱', '肇因研判子類別名稱-主要'),
    ('道路型態子類別名稱', '肇因研判子類別名稱-主要'),
    ('facility', '肇因研判子類別名稱-主要'),
    ('號誌-號誌種類名稱', '肇因研判子類別名稱-主要'),
    # ('肇因研判子類別名稱-主要', '事故類型及型態子類別名稱'),
]

# This is for english version
# from config import feature_name_map
# parent = [feature_name_map.get(col, col) for col in parent]
# cause = [feature_name_map.get(col, col) for col in cause]
# result = [feature_name_map.get(col, col) for col in result]
# white_list = [
#     (feature_name_map.get(src, src), feature_name_map.get(dst, dst))
#     for src, dst in white_list
# ]
# data = data.rename(columns=feature_name_map)

black_list = []
# cause -> parent
black_list += [(c, p) for c in cause for p in parent]
# result -> parent/cause
black_list += [(r, x) for r in result for x in (parent + cause + result)]
# 保險：parent -> result（避免直接 shortcut，如果只想透過肇因解釋）
# black_list += [(p, r) for p in parent for r in result]

In [ ]:
data = data1
# 學哪些變數之間有邊，結果是一個DAG
model = bn.structure_learning.fit(data, methodtype='hc', scoretype='bic', bw_list_method='edges',
                                  # 肇因對於事故類型一定是上游。ex. 不會因為撞路樹而造成患病，而是因為患病才造成撞路樹
                                   black_list=black_list, white_list=white_list,
                                   fixed_edges=white_list, max_indegree=None)
# 計算每個節點的 條件機率表 (CPT, Conditional Probability Table)
model_param = bn.parameter_learning.fit(model, data, scoretype='bdeu', methodtype='bayes')
# 計算邊緣強度，如果p小於顯著就是有相關
model_independence = bn.independence_test(model_param, data, test='chi_square', prune=True)

# model_independence['independence_test']
# bn.get_parents(model['model_edges'])

In [ ]:
from matplotlib import rcParams
rcParams['font.sans-serif'] = ['Microsoft JhengHei']
rcParams['axes.unicode_minus'] = False

# G = bn.plot(model, interactive=False, node_color="#36AA5B", edge_labels=None)
bn.plot(model_independence, interactive=False, edge_labels='pvalue', 
        params_static={'layout': 'spring_layout', 'font_color': "#974848", 'edge_color': "#974848"})

In [ ]:
import numpy as np
from utils_behaviour import draw_bn_plotly

draw_bn_plotly(model_independence, layout_algo='spring', en=False, width=600, height=400, seed=42, iter=30)

## Inference
這個方法針對特定的推論得出cpt，現在討論反向所以evidence會是肇因，討論特定特徵下不同設計的機率

In [ ]:
from utils_behaviour import cpd_add_n, filter_cpd_for_hotspot, get_outlier
from config import category_value_map, feature_name_map
parent = ['速限-第1當事者', '道路類別-第1當事者-名稱', '道路型態子類別名稱', 'facility', '號誌-號誌種類名稱']
child = '肇因研判子類別名稱-主要'
evidence_v = list(data['肇因研判子類別名稱-主要'].value_counts().head(5).index)

In [ ]:
for v in evidence_v:
    q2 = bn.inference.fit(
        model_param, 
        variables=parent,
        evidence={'肇因研判子類別名稱-主要': v})

    model = q2.df
    evidence_df = data[data['肇因研判子類別名稱-主要'] == v]

    filtered_condition = cpd_add_n(parent, child, model, evidence_df, cpd=False, threshold=30)
    final_filtered = filtered_condition[filtered_condition['p'] >= 0.01]
    final_filtered = final_filtered[['號誌-號誌種類名稱', '速限-第1當事者', '道路型態子類別名稱', 'facility', 'p', 'n', '道路類別-第1當事者-名稱']]
    final_filtered = final_filtered.sort_values(by=['號誌-號誌種類名稱', '速限-第1當事者', '道路型態子類別名稱', 'facility', 'p'], ascending=True)

    final_filtered.to_csv(f'../ComputedData/Behaviour/1_{v}.csv', encoding='utf-8', index=False)

### Full

In [ ]:
parent_filtered = ['道路型態子類別名稱', '號誌-號誌種類名稱']

for i in range(len(evidence_v)):
    condition_0 = pd.read_csv(f'../ComputedData/Behaviour/0_{evidence_v[i]}.csv', encoding='utf-8')
    condition_1 = pd.read_csv(f'../ComputedData/Behaviour/1_{evidence_v[i]}.csv', encoding='utf-8')
    condition_concat = pd.concat([condition_0, condition_1], axis=0)
    condition_concat = filter_cpd_for_hotspot(condition_concat)
    final_filtered = condition_concat.sort_values(
        by=['號誌-號誌種類名稱', '速限-第1當事者', '道路型態子類別名稱', 'facility', 'p'],
        ascending=True)
    # en version
    for col in parent_filtered:
        final_filtered[col] = final_filtered[col].map(category_value_map[col])
    final_filtered.rename(columns=feature_name_map, inplace=True)
    final_filtered.to_excel(f'../ComputedData/Behaviour/full_{evidence_v[i]}.xlsx', encoding='utf-8', index=False)


### Outlier Analysis

In [ ]:
outlier_data = pd.DataFrame()
fac = 0
parent_filtered = ['道路型態子類別名稱', '號誌-號誌種類名稱', '肇因研判子類別名稱-主要']

for i in range(len(evidence_v)):
    try:
        print(evidence_v[i])
        # Inference方面n和p不會是正比的主要原因是p是由cpt反推，所以不是基於n來計算
        filtered_condition = pd.read_csv(f'../ComputedData/Behaviour/{fac}_{evidence_v[i]}.csv', encoding='utf-8')
        # 已經篩選過後才存unfilter，所以不能用filter_cpd_for_hotspot
        new_con_dist = filter_cpd_for_hotspot(filtered_condition)
        new_con_dist_out = get_outlier(filtered_condition, new_con_dist)
        new_con_dist_out['肇因研判子類別名稱-主要'] = evidence_v[i]
        outlier_data = pd.concat([outlier_data, new_con_dist_out], axis=0)

        outlier_data = outlier_data[['肇因研判子類別名稱-主要', '號誌-號誌種類名稱', '速限-第1當事者', '道路型態子類別名稱', 'facility', 'p', 'n']]
        outlier_data = outlier_data.sort_values(by=['肇因研判子類別名稱-主要', '號誌-號誌種類名稱', '速限-第1當事者', '道路型態子類別名稱', 'facility', 'p'], ascending=True)
    except Exception as e:
        print(f"Error processing {evidence_v[i]}: {e}")

# en version
for col in parent_filtered:
    outlier_data[col] = outlier_data[col].map(category_value_map[col])
outlier_data.rename(columns=feature_name_map, inplace=True)

outlier_data.to_excel(f'../ComputedData/Behaviour/outlier_{fac}.xlsx', index=False)

### Hotspot Ratio Analysis

In [41]:
def make_df(prefix, evidence_v):
    data_sum = {}
    for i in range(len(evidence_v)):
        condition = pd.read_csv(f'../ComputedData/Behaviour/{prefix}_{evidence_v[i]}.csv', encoding='utf-8')
        data = filter_cpd_for_hotspot(condition)
        data_sum[evidence_v[i]] = round(data['p'].sum(), 4)

    df_sum = pd.DataFrame(
        list(data_sum.items()), 
        columns=['肇因研判子類別名稱-主要', 'p']
    )
    df_sum['facility'] = prefix
    return df_sum

df0 = make_df("0", evidence_v)
df1 = make_df("1", evidence_v)
final_df = pd.concat([df0, df1], ignore_index=True)

final_df.sort_values(by=['肇因研判子類別名稱-主要'], ascending=True, inplace=True)
final_df = final_df[['肇因研判子類別名稱-主要', 'facility', 'p']]

for col in ['肇因研判子類別名稱-主要']:
    final_df[col] = final_df[col].map(category_value_map[col])
final_df.rename(columns=feature_name_map, inplace=True)

final_df.to_excel(f'../ComputedData/Behaviour/final_sum.xlsx', index=False)

## CPD
沒有針對特徵，回傳肇因的因

In [ ]:
CPDs = bn.print_CPD(model_param)
dfprob_cause = CPDs[child]
filtered = cpd_add_n(parent, child, dfprob_cause, data, threshold=0)

In [ ]:
filtered_n = filtered[(filtered['p'] > 0.1) &
                      (filtered['n'] > 30)]
filtered_n['facility'] = filtered_n['facility'].astype(int)
final_filtered = filter_cpd(filtered_n)
final_filtered.shape

In [ ]:
final_filtered = final_filtered[['號誌-號誌種類名稱', '速限-第1當事者', '道路型態子類別名稱', 
                'facility', '肇因研判子類別名稱-主要', 'p', 'n']]
final_filtered = final_filtered.sort_values(by=['號誌-號誌種類名稱', '速限-第1當事者', 
                               '道路型態子類別名稱', 'facility', 'p'], ascending=True)

In [ ]:
from openpyxl import load_workbook

path = "../BNtables/final_filtered.xlsx"
final_filtered.to_excel(path, index=False)
wb = load_workbook(path)
ws = wb.active

for i in range(1, 4):
    col_idx = i  # 1st column
    merge_start = 2  # start from row 2 (header is in row 1)
    current_value = ws.cell(row=merge_start, column=col_idx).value

    for row in range(merge_start + 1, ws.max_row + 2):  # +2 是為了最後一個 flush
        value = ws.cell(row=row, column=col_idx).value if row <= ws.max_row else None

        if value != current_value:
            # 合併區間
            if row - merge_start > 1:
                ws.merge_cells(start_row=merge_start, start_column=col_idx,
                            end_row=row-1, end_column=col_idx)
                # 垂直置中
                ws.cell(row=merge_start, column=col_idx).alignment = ws.cell(row=merge_start, column=col_idx).alignment.copy(vertical="center", horizontal="center")
            # 更新起點
            merge_start = row
            current_value = value

    wb.save(path)


### 確認CPT的機率

In [ ]:
import numpy as np
# parent 組合數
q = dfprob_cause.groupby(parent, dropna=False).ngroups
assert np.prod([data[col].nunique() for col in parent])==q
# 每個 parent 組合底下的機率和都應該 ≈ 1
chk = dfprob_cause.groupby(parent, dropna=False)['p'].sum().unique()